In [1]:
!pip3 install pytorch-lightning

In [2]:
from argparse import ArgumentParser
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
import pytorch_lightning as pl

from data import WikiText2DataModule
from models.transformer import LMModel

In [3]:
pl.seed_everything(58)

Global seed set to 58


58

In [5]:
# ------------
# data
# ------------
data_module = WikiText2DataModule(
    train_batch_size=64,
    val_batch_size=64,
    seq_length= 64,
    vocab_size=15000
)
data_module.prepare_data()

# ------------
# model
# ------------
model = LMModel(
    vocab_size=data_module.tokenizer.get_vocab_size(),
    d_model= 128,
    n_layers= 2,
    n_heads= 4,
    d_ff= 512,
    attention='performer'
)

# ------------
# training
# ------------
checkpoint_callback = ModelCheckpoint(monitor='val_loss')
early_stop_callback = EarlyStopping(
    monitor='perplexity',
    min_delta=0.00,
    patience=5,
    verbose=False,
    mode='min'
)

Reusing dataset wikitext (/home/marek/.cache/huggingface/datasets/wikitext/wikitext-103-raw-v1/1.0.0/aa5e094000ec7afeb74c3be92c88313cd6f132d564c7effd961c10fd47c76f20)
TRAIN TOKENIZER


In [ ]:
# ------------
# Resume from checkpoint
# ------------
#trainer = pl.Trainer(resume_from_checkpoint='./lightning_logs/version_12/checkpoints/epoch=7-step=217777.ckpt',gpus=1,
#trainer = pl.Trainer(resume_from_checkpoint='./lightning_logs/version_13/checkpoints/epoch=9-step=251031.ckpt',gpus=1, max_epochs=25, val_check_interval=500)

#trainer = pl.Trainer(resume_from_checkpoint="lightning_logs/version_50/checkpoints/epoch=4-step=115015.ckpt",gpus=1, max_epochs=100, val_check_interval=500)

trainer = pl.Trainer(callbacks=[checkpoint_callback], max_epochs=24, val_check_interval=500)
trainer.fit(model=model, datamodule=data_module)